In [ ]:
# setting up the connection between kaggle and colab

In [ ]:
pip install kaggle 

In [ ]:
mkdir ~/.kaggle

In [ ]:
cp kaggle.json ~/.kaggle

In [ ]:
! kaggle datasets download -d prasunroy/natural-images

In [ ]:
! unzip natural-images.zip

In [ ]:
# import libs

In [ ]:
import os
from tensorflow import keras

In [ ]:
# setting up the data and spliting it for test and training

In [ ]:
data_dir ='./natural_images'
batch_size = 32
img_height = 180
img_width = 180

In [ ]:
train_ds = keras.preprocessing.image_dataset_from_directory(
                                                              data_dir,
                                                              validation_split=0.2,
                                                              subset="training",
                                                              seed=123,
                                                              image_size=(img_height, img_width),
                                                              batch_size=batch_size
                                                            )

In [ ]:
val_ds = keras.preprocessing.image_dataset_from_directory(
                                                            data_dir,
                                                            validation_split=0.2,
                                                            subset="validation",
                                                            seed=123,
                                                            image_size=(img_height, img_width),
                                                            batch_size=batch_size
                                                          )

Found 6899 files belonging to 8 classes.
Using 1379 files for validation.


In [ ]:
# validating our data is corrextly retrived 
class_names = train_ds.class_names
print(class_names)

['airplane', 'car', 'cat', 'dog', 'flower', 'fruit', 'motorbike', 'person']
